In [1]:
#importar as bibliotecas
import pandas as pd
import numpy as np
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [2]:
nltk.download('stopwords')
stoplist = set(stopwords.words("english"))
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\AvecomDev\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
#carregar dataset
#os dados foram carregados do kaggle para treinar nossa ia.
df = pd.read_csv('train.csv',encoding='utf8')

In [4]:
df.isnull().sum()
#retirar nulos
df.dropna(inplace=True)
df.isnull().sum()

id        0
title     0
author    0
text      0
label     0
dtype: int64

In [5]:
#juntar os campos
df['all'] = df['title'] +'  '+ df['author'] +'  '+ df['text']  
pd.options.display.max_colwidth = 150
print(df['all'])

0        House Dem Aide: We Didn’t Even See Comey’s Letter Until Jason Chaffetz Tweeted It  Darrell Lucus  House Dem Aide: We Didn’t Even See Comey’s Lette...
1        FLYNN: Hillary Clinton, Big Woman on Campus - Breitbart  Daniel J. Flynn  Ever get the feeling your life circles the roundabout rather than heads ...
2        Why the Truth Might Get You Fired  Consortiumnews.com  Why the Truth Might Get You Fired October 29, 2016 \nThe tension between intelligence analy...
3        15 Civilians Killed In Single US Airstrike Have Been Identified  Jessica Purkiss  Videos 15 Civilians Killed In Single US Airstrike Have Been Iden...
4        Iranian woman jailed for fictional unpublished story about woman stoned to death for adultery  Howard Portnoy  Print \nAn Iranian woman has been s...
                                                                                 ...                                                                          
20795    Rapper T.I.: Trump a ’Poster Child Fo

In [6]:
#criar função de stemmer
#ele tenta trazer a palavra até a palavra rais, ou seja, se a palavra for atuando ele tenta trazer ela para atuar.
porter_stemmer = PorterStemmer()
 
def stemming_tokenizer(df):

    words= re.sub(r"[^A-Za-z]", " ", df).lower().split()
    words= [porter_stemmer.stem(word) for word in words if not words in stopwords.words('english')]
    words= ' '.join(words)
    return words 

In [7]:

df['features'] = df['all'].apply(stemming_tokenizer)
print(df['features'])

0        hous dem aid we didn t even see comey s letter until jason chaffetz tweet it darrel lucu hous dem aid we didn t even see comey s letter until jaso...
1        flynn hillari clinton big woman on campu breitbart daniel j flynn ever get the feel your life circl the roundabout rather than head in a straight ...
2        whi the truth might get you fire consortiumnew com whi the truth might get you fire octob the tension between intellig analyst and polit policymak...
3        civilian kill in singl us airstrik have been identifi jessica purkiss video civilian kill in singl us airstrik have been identifi the rate at whic...
4        iranian woman jail for fiction unpublish stori about woman stone to death for adulteri howard portnoy print an iranian woman ha been sentenc to si...
                                                                                 ...                                                                          
20795    rapper t i trump a poster child for w

In [341]:
#separa em features e label
#label representa a veracidade da informação sendo 1 para fake e 0 pra real
x = df['features'].values
y = df['label'].values

x.size

18285

In [342]:
# convert de texto para numeros
vectorizer = TfidfVectorizer()
vectorizer.fit(x)
z = vectorizer.transform(x)


In [362]:
z_train, z_test, y_train, y_test = train_test_split(z, y, test_size=0.2,stratify=y, random_state=42) 

In [363]:
print(len(y_test))

3657


In [364]:
#treinando o modelo

lr = LogisticRegression()
lr.fit(z_test,y_test)

LogisticRegression()

In [378]:
#previsão 

pred = lr.predict(z_test)

acc = accuracy_score(y_test,pred)
f'Acurácia:{acc * 100:.2f}'


'Acurácia:97.10'

In [381]:
# Pré-processamento da notícia
noticia = "Rapper T.I.: Trump a ’Poster Child For White Supremacy’  Jerome Hudson  Rapper T. I. unloaded on black celebrities who met with Donald Trump afte"
noticia_preprocessada = stemming_tokenizer(noticia)

# Convertendo a notícia em uma representação numérica
noticia_numerica = vectorizer.transform([noticia_preprocessada])

#Fazendo a previsão
probabilidade_falsa = lr.predict_proba(noticia_numerica)[:, 1]  # Obtém a probabilidade da classe falsa

chance_falsa = probabilidade_falsa[0] * 100  # Multiplica por 100 para obter a porcentagem
chance_verdadeira = 100 - chance_falsa  # Calcula a porcentagem de chances de ser verdadeira

if chance_falsa > 50:
    print("A notícia é falsa.")
else:
    print("A notícia é verdadeira.")



A notícia é falsa.
